In [ ]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class PrepareCallbackConfig:
    root_dir: Path
    tensorboard_root_log_dir: Path
    checkpoint_model_filepath: Path
    

In [ ]:
from src.deepClassifier.constants import *
from src.deepClassifier.utils import read_yaml, create_directories

In [ ]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])
        
    def get_prepare_callback_config(self) -> PrepareBaseModelConfig:
        config = self.config.prepare_callbacks
        model_ckpt_dir = os.path.dirname(config.checkpoint_model_filepath)
        
        # this will create base directories
        create_directories([
            Path(model_ckpt_dir),
            Path(config.tensorboard_root_log_dir)
            ])
        
         prepare_callback_config = PrepareBaseModelConfig(
            root_dir=Path(config.root_dir),
            tensorboard_root_log_di=Path(config.tensorboard_root_log_dir),
            checkpoint_model_filepath=Path(config.checkpoint_model_filepath)
        )

        return prepare_callback_config
            

In [ ]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time

class PrepareCallback:
    def __init__(self, config: PrepareCallbackConfig):
        self.config=config
    
    @property
    def _create_tb_callbacks(self):
        
        """ this will keep crating new name for tensor log dir folder"""
        
        timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
        tb_running_log_dir = os.path.join(
            self.config.tensorbord_root_log_dir,
            f"tb_log_at_{timestamp}"
        )
        return tf.keras.callbacks.TensorBoard(Log_dir=tb_running_log_dir)
    
    
    @property
    def _create_ckpt_callbacks(self):
        return tf.keras.callbacks.ModelCheckpoint(
            filepath=self.config.checkpoint_model_filepath,
            save_best_only=True
        )
        
        
    def get_tb_ckpt_callbacks(self):
        return [
            self._create_tb_callbacks,
            self._create_ckpt_callbacks
        ]
    
   

In [ ]:
try:
    config = ConfigurationManager()
    prepare_callback_config = config.get_prepare_callback_config()
    prepare_callback = PrepareCallBack(config=prepare_callback_config)
    prepare_callback.get_tb_ckpt_callbacks()
except Exception as e:
    raise e

In [3]:
import time
time.strftime("%Y-%m-%d::%H-%M-%S")

'2022-09-23::20-42-13'